<a href="https://colab.research.google.com/github/henrijoel1209/FORUM/blob/main/Exercice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
from google.colab import drive

def charger_donnees():
    """Charge et fusionne les données de population et surface"""
    try:
        # Monter Google Drive
        drive.mount('/content/drive', force_remount=False)

        base_path = '/content/drive/My Drive/Exercice/DATA/'

        # =========================
        # Charger population (Excel)
        # =========================
        df_pop = pd.read_excel(
            base_path + 'Populations_CI_2019.xlsx',
            sheet_name='CIV_ADM1_POP'
        )

        # Nettoyage
        df_pop = df_pop.dropna(subset=['ADM1_FR'])
        df_pop = df_pop[df_pop['ADM1_FR'] != '#adm1+name']

        # Conversion population en numérique
        df_pop['T_TL'] = pd.to_numeric(df_pop['T_TL'], errors='coerce')

        # =========================
        # Charger surface (GeoJSON)
        # =========================
        with open(base_path + 'extracted/civ_admin1.geojson', 'r', encoding='utf-8') as f:
            geo_data = json.load(f)

        surface_data = []
        for feature in geo_data['features']:
            props = feature['properties']
            surface_data.append({
                'region': props['adm1_name'],
                'surface': props['area_sqkm']
            })

        df_surface = pd.DataFrame(surface_data)

        # =========================
        # Fusionner les données
        # =========================
        df = pd.merge(df_pop, df_surface, left_on='ADM1_FR', right_on='region')

        # Calcul densité
        df['densite'] = df['T_TL'] / df['surface']

        # Colonnes finales
        df_final = df[['ADM1_FR', 'T_TL', 'surface', 'densite']].rename(columns={
            'ADM1_FR': 'region',
            'T_TL': 'population',
            'surface': 'surface_km2'
        })

        return df_final

    except Exception as e:
        print("Erreur détectée :", e)  # Affiche l’erreur réelle
        return None

def main():
    data = charger_donnees()

    if data is None:
        print("Erreur: Impossible de charger les données")
        return

    # Tri UNE SEULE FOIS
    regions = sorted(data['region'])

    while True:
        print("\n" + "="*50)
        print("CALCUL DE DENSITE DE POPULATION")
        print("="*50)

        # Afficher régions
        for i, region in enumerate(regions, 1):
            print(f"{i:2d}. {region}")

        choix = input("\nChoisissez une region (numero ou nom): ").strip()

        # =========================
        # Trouver la région
        # =========================
        if choix.isdigit():
            idx = int(choix) - 1

            if 0 <= idx < len(regions):
                region_choisie = regions[idx]
            else:
                print("Numero invalide")
                continue
        else:
            if choix in data['region'].values:
                region_choisie = choix
            else:
                print("Region non trouvee")
                continue

        # =========================
        # Afficher résultats
        # =========================
        result = data[data['region'] == region_choisie].iloc[0]

        print(f"\nRegion: {result['region']}")
        print(f"Population: {result['population']:,.0f} habitants")
        print(f"Surface: {result['surface_km2']:,.2f} km²")
        print(f"Densite: {result['densite']:,.2f} habitants/km²")

        if input("\nAutre region? (o/n): ").lower() != 'o':
            break

if __name__ == "__main__":
    main()